<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/wavelets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tsfresh
import pywt
import pandas as pd
import numpy as np

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1ps_Bh93KQRJ4WbnilORmKd1BaWUXtmt1' # files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 400hz
Processing folder: 100hz
Processing folder: 800hz


In [3]:
import gc
del folder_list, file_list, folder, index, file
gc.collect()

1597

In [4]:
train_target = pd.read_csv('800hz-train-target-20percent-1000files.csv')
train_target.drop(columns=['Unnamed: 0'], inplace = True)
test_target = pd.read_csv('800hz-test-target-20percent-1000files.csv')
test_target.drop(columns=['Unnamed: 0'], inplace = True)
train_feature = pd.read_csv('800hz-train-feature-20percent-1000files.csv')
train_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color'], inplace = True)
test_feature = pd.read_csv('800hz-test-feature-20percent-1000files.csv')
test_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color' ], inplace = True)

In [5]:
train_feature

,Uhrzeit,I_In,V_In,I_Out(A),V_Out(V),Temp(°C),fileno
0,2024-08-30 18:12:02.311352:311,0.31,8.75,0.29,9.03,35.13,0
1,2024-08-30 18:12:02.312303:312,0.35,8.79,0.25,8.97,35.13,0
2,2024-08-30 18:12:02.313072:313,0.35,8.79,0.25,8.97,35.13,0
3,2024-08-30 18:12:02.315592:315,0.35,8.79,0.25,8.97,35.13,0
4,2024-08-30 18:12:02.316626:316,0.35,8.79,0.25,8.97,35.13,0
...,...,...,...,...,...,...,...
4670961,2024-09-03 17:19:53.916664:916,0.31,8.75,0.29,9.00,33.87,3199
4670962,2024-09-03 17:19:53.919914:919,0.37,8.70,0.23,9.00,33.87,3199
4670963,2024-09-03 17:19:53.920314:920,0.37,8.70,0.23,9.00,33.87,3199
4670964,2024-09-03 17:19:53.920656:920,0.37,8.70,0.23,9.00,33.87,3199


In [6]:
wavelet_type = 'db4'
decomp_level = 4

In [7]:
x_train_sorted = train_feature.sort_values(by=['fileno', 'Uhrzeit'])
x_test_sorted = test_feature.sort_values(by=['fileno', 'Uhrzeit'])

train_grouped = x_train_sorted.groupby('fileno')
test_grouped = x_test_sorted.groupby('fileno')

In [8]:
import gc
del x_train_sorted, x_test_sorted, train_feature, test_feature
gc.collect()

300

In [9]:
def wavelet_transform(grouped):
  wavelet_df_I_In = {}
  wavelet_df_I_Out = {}
  wavelet_df_V_In = {}
  wavelet_df_V_Out = {}
  wavelet_df_Temp = {}

  wavelet_features_I_In = []
  wavelet_features_I_Out =[]
  wavelet_features_V_In =[]
  wavelet_features_V_Out =[]
  wavelet_features_Temp =[]


  for fileno, group in grouped:
      signal_I_In = group['I_In'].values
      signal_V_In = group['V_In'].values
      signal_I_Out = group['I_Out(A)'].values
      signal_V_Out = group['V_Out(V)'].values
      signal_Temp = group['Temp(°C)'].values

      # wavelet decomposition
      coeffs_I_In = pywt.wavedec(signal_I_In, wavelet_type, level=decomp_level)
      coeffs_V_In = pywt.wavedec(signal_V_In, wavelet_type, level=decomp_level)
      coeffs_I_Out = pywt.wavedec(signal_I_Out, wavelet_type, level=decomp_level)
      coeffs_V_Out = pywt.wavedec(signal_V_Out, wavelet_type, level=decomp_level)
      coeffs_Temp = pywt.wavedec(signal_Temp, wavelet_type, level=decomp_level)

  # for I_In

      # Extract features from coefficients
      features_I_In = {}
      for i, coeff in enumerate(coeffs_I_In):
          if len(coeff) == 0:
              continue
          # features: energy, entropy, variance
          features_I_In[f'I_In_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_I_In[f'I_In_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_I_In[f'I_In_wavelet_var_level_{i}'] = np.var(coeff)

      # Add fileno to map features back
      features_I_In['fileno'] = fileno
      wavelet_features_I_In.append(features_I_In)


  #for I_Out
      features_I_Out = {}
      for i, coeff in enumerate(coeffs_I_Out):
          if len(coeff) == 0:
              continue
          # features: energy, entropy,
          features_I_Out[f'I_Out_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_I_Out[f'I_Out_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_I_Out[f'I_Out_wavelet_var_level_{i}'] = np.var(coeff)

      wavelet_features_I_Out.append(features_I_Out)



  #for V_In
      features_V_In = {}
      for i, coeff in enumerate(coeffs_V_In):
          if len(coeff) == 0:
              continue
          # features: energy, entropy,
          features_V_In[f'V_In_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_V_In[f'V_In_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_V_In[f'V_In_wavelet_var_level_{i}'] = np.var(coeff)

      wavelet_features_V_In.append(features_V_In)


  #for V_Out
      features_V_Out = {}
      for i, coeff in enumerate(coeffs_V_Out):
          if len(coeff) == 0:
              continue
          # features: energy, entropy,
          features_V_Out[f'V_Out_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_V_Out[f'V_Out_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_V_Out[f'V_Out_wavelet_var_level_{i}'] = np.var(coeff)

      wavelet_features_V_Out.append(features_V_Out)

  #for Temp
      features_Temp = {}
      for i, coeff in enumerate(coeffs_Temp):
          if len(coeff) == 0:
              continue
          # features: energy, entropy,
          features_Temp[f'Temp_wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features_Temp[f'Temp_wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2 + 1e-10))
          features_Temp[f'Temp_wavelet_var_level_{i}'] = np.var(coeff)

      wavelet_features_Temp.append(features_Temp)

  # Convert to DataFrame
  wavelet_df_I_In = pd.DataFrame(wavelet_features_I_In)
  wavelet_df_I_Out = pd.DataFrame(wavelet_features_I_Out)
  wavelet_df_V_In = pd.DataFrame(wavelet_features_V_In)
  wavelet_df_V_Out = pd.DataFrame(wavelet_features_V_Out)
  wavelet_df_Temp = pd.DataFrame(wavelet_features_Temp)


  wavelet_df = pd.concat([wavelet_df_I_In, wavelet_df_I_Out, wavelet_df_V_In, wavelet_df_V_Out, wavelet_df_Temp], axis=1)
  wavelet_df = wavelet_df.set_index('fileno')
  return wavelet_df


In [10]:
wavelet_df_train = wavelet_transform(train_grouped)
wavelet_df_test = wavelet_transform(test_grouped)

In [11]:
wavelet_df_train.isna().sum()

,0
I_In_wavelet_energy_level_0,0
I_In_wavelet_entropy_level_0,0
I_In_wavelet_var_level_0,0
I_In_wavelet_energy_level_1,0
I_In_wavelet_entropy_level_1,0
...,...
Temp_wavelet_entropy_level_3,0
Temp_wavelet_var_level_3,0
Temp_wavelet_energy_level_4,0
Temp_wavelet_entropy_level_4,0


In [12]:
wavelet_df_test.isna().sum()

,0
I_In_wavelet_energy_level_0,0
I_In_wavelet_entropy_level_0,0
I_In_wavelet_var_level_0,0
I_In_wavelet_energy_level_1,0
I_In_wavelet_entropy_level_1,0
...,...
Temp_wavelet_entropy_level_3,0
Temp_wavelet_var_level_3,0
Temp_wavelet_energy_level_4,0
Temp_wavelet_entropy_level_4,0


In [13]:
wavelet_df_test

,I_In_wavelet_energy_level_0,I_In_wavelet_entropy_level_0,I_In_wavelet_var_level_0,I_In_wavelet_energy_level_1,I_In_wavelet_entropy_level_1,I_In_wavelet_var_level_1,I_In_wavelet_energy_level_2,I_In_wavelet_entropy_level_2,I_In_wavelet_var_level_2,I_In_wavelet_energy_level_3,...,Temp_wavelet_var_level_1,Temp_wavelet_energy_level_2,Temp_wavelet_entropy_level_2,Temp_wavelet_var_level_2,Temp_wavelet_energy_level_3,Temp_wavelet_entropy_level_3,Temp_wavelet_var_level_3,Temp_wavelet_energy_level_4,Temp_wavelet_entropy_level_4,Temp_wavelet_var_level_4
fileno,,,,,,,,,,,,,,,,,,,,,
200,746.681796,-761.417725,0.099850,4.715858,12.851561,0.015878,3.599270,11.271322,0.006121,1.208338,...,0.070714,22.803709,49.183828,0.038722,7.510646,28.165621,0.006423,5.500766,23.976731,0.002360
201,729.110034,-761.770733,0.107419,5.227221,13.709412,0.018341,2.899759,11.946577,0.005149,1.202051,...,0.063695,19.629014,42.963234,0.034855,6.843425,25.547298,0.006101,5.077448,21.717912,0.002270
202,817.519041,-810.155849,0.091066,5.516826,15.615351,0.016584,3.455399,12.393426,0.005259,1.672316,...,0.063454,26.119232,56.211614,0.039752,8.420402,31.216648,0.006439,6.168688,26.589752,0.002364
203,720.040036,-736.112176,0.104052,5.248435,11.518881,0.018277,3.509810,11.098922,0.006173,1.103427,...,0.081707,23.566815,52.180911,0.041491,8.697609,32.517556,0.007701,5.817050,25.261819,0.002584
204,754.288939,-767.016951,0.095378,4.997413,13.039681,0.016656,3.348494,11.349758,0.005636,1.411598,...,0.068749,20.210195,45.649664,0.033950,7.278772,28.013933,0.006157,5.237741,22.668171,0.002221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,678.416291,-344.859963,0.040524,4.086222,12.749115,0.009380,3.276804,14.733753,0.003793,1.076451,...,0.063482,31.774313,71.509381,0.036768,10.046390,38.992567,0.005832,7.546461,33.839335,0.002193
3996,667.684731,-322.493529,0.036729,3.532805,13.768944,0.008046,4.501818,16.254766,0.005184,1.104827,...,0.081818,34.558790,71.337442,0.039855,11.894874,42.880404,0.006883,8.666969,36.479428,0.002513
3997,679.450253,-327.571473,0.035847,4.191070,13.048022,0.009414,3.131279,14.779142,0.003554,1.097801,...,0.077368,33.282186,75.230056,0.037717,11.338130,42.273007,0.006457,8.316367,35.946923,0.002373


In [14]:
train_target

,color
0,0
1,0
2,0
3,0
4,0
...,...
795,3
796,3
797,3
798,3


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score

In [16]:
import sklearn

In [17]:
model2 = RandomForestClassifier()
model2.fit(wavelet_df_train, train_target)
y_pred = model2.predict(wavelet_df_test)
accura = sklearn.metrics.accuracy_score(test_target, y_pred)
print("Accuracy ",accura)
precision_score = sklearn.metrics.precision_score(test_target, y_pred, average='weighted')
print("Precision", precision_score)
# print(classification_report(test_target, y_pred))
cm = confusion_matrix(test_target, y_pred)
print("Confusion Matrix:")
print(cm)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy  0.76
Precision 0.7460538332831043
Confusion Matrix:
[[797   0   2   1]
 [  0 351 231 218]
 [ 12 290 497   1]
 [  0  13   0 787]]


In [18]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score

final_model = xgb.XGBClassifier(
    # objective="multi:softmax",
    # random_state=42,
    alpha=10,
    reg_lambda=1,
    # max_depth=10,
    min_child_weight=10,
    # subsample=0.8,
    colsample_bytree=0.8
)

final_model.fit(wavelet_df_train, train_target)

y_pred_final = final_model.predict(wavelet_df_test)

final_accuracy = accuracy_score(test_target, y_pred_final)
final_precision = precision_score(test_target, y_pred_final, average='weighted')
final_cm = confusion_matrix(test_target, y_pred_final)

print(f"Final Model Accuracy: {final_accuracy:.4f}")
print(f"Final Model Precision: {final_precision:.4f}")
# print(classification_report(test_target, y_pred_final))
print("Final Model Confusion Matrix:")
print(final_cm)

Final Model Accuracy: 0.6784
Final Model Precision: 0.6469
Final Model Confusion Matrix:
[[797   0   2   1]
 [ 72 349 273 106]
 [ 94 470 234   2]
 [  0   9   0 791]]


In [19]:
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

def xgb_(X, y, num_folds=5):

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=24)

    f1_scores = []
    accuracies = []
    precisions = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Initialize and train the XGBoost model with regularization
        xgb1 = xgb.XGBClassifier(
            objective="multi:softmax",
            random_state=42,
            alpha=0.1,
            reg_lambda=0.1,
            max_depth=4,
            min_child_weight=10,
            subsample=0.8,
            colsample_bytree=0.8
        )

        # Fit the model
        xgb1.fit(X_train.to_numpy(), y_train.to_numpy().ravel())

        # Make predictions
        y_pred = xgb1.predict(X_test.to_numpy())

        # Ensure y_test is 1D for comparison
        y_test = y_test.to_numpy().ravel().astype(int)
        y_pred = y_pred.astype(int)

        # Calculate and store F1-score for this fold
        f1 = f1_score(y_test, y_pred, average='weighted')
        f1_scores.append(f1)

        # Calculate and store accuracy and precision for this fold
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        precision = precision_score(y_test, y_pred, average='weighted')
        precisions.append(precision)

        print(classification_report(y_test, y_pred))
        print(f"Fold {fold_idx + 1} F1-score: {f1:.4f}")
        print(f"Fold {fold_idx + 1} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold_idx + 1} Precision: {precision:.4f}")

        # Calculate and print confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("Confusion Matrix:")
        print(cm)

        print("-" * 30)

    # Calculate and print average metrics across all folds
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)

    print(f"Average F1-score across {num_folds} folds: {avg_f1:.4f}")
    print(f"Average Accuracy across {num_folds} folds: {avg_accuracy:.4f}")
    print(f"Average Precision across {num_folds} folds: {avg_precision:.4f}")

    # Final model: Train on the entire dataset
    print("Training final model on the entire dataset...")
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)

    # Final model with regularization
    final_model = xgb.XGBClassifier(
        objective="multi:softmax",
        random_state=42,
        alpha=1,
        reg_lambda=1,
        # max_depth=5,
        min_child_weight=20,
        # subsample=0.8,
        # colsample_bytree=0.8
    )

    # Train on the entire dataset
    final_model.fit(X_train_final.to_numpy(), y_train_final.to_numpy().ravel())

    # Evaluate the final model on the test set
    y_pred_final = final_model.predict(X_test_final.to_numpy())

    # Calculate accuracy, precision, confusion matrix for final model
    final_accuracy = accuracy_score(y_test_final, y_pred_final)
    final_precision = precision_score(y_test_final, y_pred_final, average='weighted')
    final_cm = confusion_matrix(y_test_final, y_pred_final)

    print(f"Final Model Accuracy: {final_accuracy:.4f}")
    print(f"Final Model Precision: {final_precision:.4f}")
    print("Final Model Confusion Matrix:")
    print(final_cm)

    # Save the final model
    # with open('XGB_final_model.pkl', 'wb') as f:
    #     pickle.dump(final_model, f)
    # files.download('XGB_final_model.pkl')



In [20]:
xgb_ (pd.concat([wavelet_df_train, wavelet_df_test]), pd.concat([train_target, test_target]))

Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       210
           1       0.97      0.97      0.97       194
           2       0.97      0.98      0.98       199
           3       1.00      0.99      0.99       197

    accuracy                           0.99       800
   macro avg       0.99      0.99      0.99       800
weighted avg       0.99      0.99      0.99       800

Fold 1 F1-score: 0.9863
Fold 1 Accuracy: 0.9862
Fold 1 Precision: 0.9864
Confusion Matrix:
[[209   0   1   0]
 [  0 189   5   0]
 [  0   3 196   0]
 [  0   2   0 195]]
------------------------------
Fold 2:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       201
           1       0.97      0.97      0.97       191
           2       0.99      0.97      0.98       208
           3       0.99      1.00      0.99       200

    accuracy                           0.98       800
   macro avg       0.98 